In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import time


In [16]:
Data =pd.read_csv("housing.csv")
X = Data.drop(['median_house_value'] , axis=1)
Y = Data['median_house_value']
Y = Y.values.reshape(-1,1)


Place=['INLAND','ISLAND','NEAR BAY','<1H OCEAN','NEAR OCEAN']

encoder = OrdinalEncoder(
    categories=[Place],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)
cat_cols = ['ocean_proximity']
new_cols = ['Place']
encoder.fit(X[cat_cols])

X[new_cols] = encoder.transform(X[cat_cols])

X = X.select_dtypes(include=['number'])

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15,random_state=42)

median_value_train = X_train.median(numeric_only = True)
X_train = X_train.fillna(median_value_train)

median_value_test =  X_test.median(numeric_only = True)
X_test = X_test.fillna(median_value_test)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
y_scaler = StandardScaler()
y_scaler.fit(Y_train)
Y_train = y_scaler.transform(Y_train)



In [17]:
X_train_tensor = torch.tensor(X_train,dtype = torch.float32)
X_test_tensor = torch.tensor(X_test,dtype = torch.float32)
Y_train_tensor = torch.tensor(Y_train,dtype = torch.float32).view(-1,1)
Y_test_tensor = torch.tensor(Y_test,dtype = torch.float32).view(-1,1)
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
train_loader = DataLoader(train_dataset,batch_size=128,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=128)

In [18]:
NUM_EPOCHS =100
RANDOM_SEED = 42
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [19]:
sclass Model(torch.nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.my_net = torch.nn.Sequential(
            torch.nn.Linear(in_features=num_features, out_features=6),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=6, out_features=4),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=4, out_features=1),


        )
    def forward(self, x):
        out = self.my_net(x)
        return out


torch.manual_seed(RANDOM_SEED)
model = Model(num_features=9)

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

def compute_loss(net,dataloader):
    curr_loss = 0
    with torch.no_grad():
        for cnt , (fet, output) in enumerate(dataloader):
            fet = fet.view((-1, 9)).to(DEVICE)
            output = output.view((-1,1)).to(DEVICE)
            out = net(fet)
            loss = F.mse_loss(out,output)
            curr_loss += loss
        return float(curr_loss)/len(dataloader)

start_time = time.time()
minibatch_cost = []
epoch_cost = []

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx , (features , targets) in enumerate(train_loader):
        features = features.view(-1,9).to(DEVICE)
        targets = targets.to(DEVICE)
        pred = model.forward(features)
        cost = F.mse_loss(pred,targets.float().view(-1,1))
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        minibatch_cost.append(cost.item())
        if not batch_idx % 64:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                   %(epoch+1, NUM_EPOCHS, batch_idx,
                     len(train_loader), cost.item()))
    cost = compute_loss(model, train_loader)
    epoch_cost.append(cost)
    print('Epoch: %03d/%03d Train Cost: %.4f' % (
            epoch+1, NUM_EPOCHS, cost))
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))











Epoch: 001/100 | Batch 000/138 | Cost: 1.1674
Epoch: 001/100 | Batch 064/138 | Cost: 1.1557
Epoch: 001/100 | Batch 128/138 | Cost: 1.2099
Epoch: 001/100 Train Cost: 1.0098
Time elapsed: 0.00 min
Epoch: 002/100 | Batch 000/138 | Cost: 0.9118
Epoch: 002/100 | Batch 064/138 | Cost: 0.7931
Epoch: 002/100 | Batch 128/138 | Cost: 1.2088
Epoch: 002/100 Train Cost: 0.9815
Time elapsed: 0.01 min
Epoch: 003/100 | Batch 000/138 | Cost: 0.9530
Epoch: 003/100 | Batch 064/138 | Cost: 0.9555
Epoch: 003/100 | Batch 128/138 | Cost: 0.9731
Epoch: 003/100 Train Cost: 0.9638
Time elapsed: 0.01 min
Epoch: 004/100 | Batch 000/138 | Cost: 0.9814
Epoch: 004/100 | Batch 064/138 | Cost: 0.8919
Epoch: 004/100 | Batch 128/138 | Cost: 0.7258
Epoch: 004/100 Train Cost: 0.9469
Time elapsed: 0.02 min
Epoch: 005/100 | Batch 000/138 | Cost: 0.9507
Epoch: 005/100 | Batch 064/138 | Cost: 0.8196
Epoch: 005/100 | Batch 128/138 | Cost: 0.8746
Epoch: 005/100 Train Cost: 0.9178
Time elapsed: 0.02 min
Epoch: 006/100 | Batch 00